美国建网

In [1]:
# import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime, timedelta

# file_path = '../美国航班数据/US_data (2015-2021, update)/2015-1.csv'
# data = pd.read_csv(file_path)

# us
def read_year_data(year):
    if year in [2015,2016]:
        df = pd.read_csv("../美国航班数据/US_data (2015-2021, update)/" + str(year) + "-1.csv")
        for j in range(2, 13):
            path = "../美国航班数据/US_data (2015-2021, update)/" + str(year) + "-" + str(j) + ".csv"
            temp = pd.read_csv(path)
            df = pd.concat([df, temp], axis=0, ignore_index=True, sort=False)
        return df
    elif year in [2017,2018,2019,2020,2021]:
        df = pd.read_csv("../美国航班数据/US_data (2015-2021, update)/" + str(year) + "-01.csv")
        for j in range(2, 10):
            path = "../美国航班数据/US_data (2015-2021, update)/" + str(year) + "-" + "0" + str(j) + ".csv"
            temp = pd.read_csv(path)
            df = pd.concat([df, temp], axis=0, ignore_index=True, sort=False)
        for j in range(10, 13):
            path = "../美国航班数据/US_data (2015-2021, update)/" + str(year) + "-" + str(j) + ".csv"
            temp = pd.read_csv(path)
            df = pd.concat([df, temp], axis=0, ignore_index=True, sort=False)
        return df
    else:
        print("invalid year")

def read_all_data():
    df = read_year_data(2015)
    for year in range(2015, 2022):
        tmp_df = read_year_data(year)
        df = pd.concat([df, tmp_df], axis=0, ignore_index=True, sort=False)
    return df

# def read_month_data(year:int, month:int, dtypes_mapping):
#     if year in [2015,2016] and month_check(month):
#         df = pd.read_csv("../美国航班数据/US_data (2015-2021, update)/" + str(year) + "-" + str(month) + ".csv", dtype=dtypes_mapping)
#         return df
#     elif year in [2017,2018,2019,2020,2021] and month_check(month):
#         df = pd.read_csv("../美国航班数据/US_data (2015-2021, update)/" + str(year) + "-" + str(month).zfill(2) + ".csv", dtype=dtypes_mapping)
#         return df
#     else:
#         print("invalid year")

# data = read_month_data(2015, 1)

data = read_year_data(2017)
data

,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,CRS_ARR_TIME,ARR_TIME,ARR_DELAY_NEW,CANCELLED,CANCELLATION_CODE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1/1/2017 12:00:00 AM,AA,N001AA,IAH,MIA,701,710.0,9.0,1025,1021.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1/1/2017 12:00:00 AM,AA,N001AA,MIA,ATL,1850,2016.0,86.0,2051,2213.0,82.0,0.0,NaN,38.0,0.0,0.0,0.0,44.0
2,1/1/2017 12:00:00 AM,AA,N002AA,DFW,GUC,1249,1244.0,0.0,1400,1400.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1/1/2017 12:00:00 AM,AA,N002AA,DFW,JAC,1845,1838.0,0.0,2041,2022.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1/1/2017 12:00:00 AM,AA,N002AA,GUC,DFW,1440,1438.0,0.0,1745,1732.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5674616,12/31/2017 12:00:00 AM,WN,N968WN,SAT,LAS,1105,1101.0,0.0,1200,1141.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5674617,12/31/2017 12:00:00 AM,WN,N969WN,BNA,DCA,1620,1711.0,51.0,1855,1930.0,35.0,0.0,NaN,8.0,0.0,0.0,0.0,27.0
5674618,12/31/2017 12:00:00 AM,WN,N969WN,DAL,MDW,1150,1157.0,7.0,1355,1412.0,17.0,0.0,NaN,7.0,0.0,10.0,0.0,0.0
5674619,12/31/2017 12:00:00 AM,WN,N969WN,MDW,BNA,1430,1516.0,46.0,1550,1629.0,39.0,0.0,NaN,25.0,0.0,0.0,0.0,14.0


In [ ]:
airport_data = pd.read_csv('../美国航班数据/美国机场.csv')
airport_codes = set(airport_data['Code'])
airport_df = data[(data['ORIGIN'].isin(airport_codes)) & (data['DEST'].isin(airport_codes))]
without_cancelled = airport_df.drop(airport_df[airport_df['CANCELLED'] == 1].index).dropna(how= 'all')

# without_cancelled = data.drop(data[data['CANCELLED'] == 1].index).dropna(how= 'all')
print(without_cancelled)
# print(without_cancelled.iloc[100000]['FL_DATE'].split(' ')[0].split('/')[0])
# without_cancelled.iloc[100000]['FL_DATE'].split(' ')[0]

In [ ]:
def parse_time(time):
    #解析时间字符串
    time_str = str(int(time)).zfill(4)
    hour = int(time_str[:2])
    minute = int(time_str[2:])
    return hour, minute

def time_difference(fl_date1, time1, fl_date2, time2):
    #计算两个时间点之间的时间差，以分钟为单位
    month1 = int(fl_date1.split(' ')[0].split('/')[0])
    date1 = int(fl_date1.split(' ')[0].split('/')[1])
    year1 = int(fl_date1.split(' ')[0].split('/')[2])
    hour1, minute1 = parse_time(time1)

    month2 = int(fl_date2.split(' ')[0].split('/')[0])
    date2 = int(fl_date2.split(' ')[0].split('/')[1])
    year2 = int(fl_date2.split(' ')[0].split('/')[2])
    hour2, minute2 = parse_time(time2)

    if hour1 == 24:
        dt1 = datetime(year1, month1, date1, 0, minute1) + timedelta(days=1)
    else:
        dt1 = datetime(year1, month1, date1, hour1, minute1)
    
    if hour2 == 24:
        dt2 = datetime(year2, month2, date2, 0, minute2) + timedelta(days=1)
    else:
        dt2 = datetime(year2, month2, date2, hour2, minute2)
    
    # 计算时间差
    delta = dt2 - dt1
    difference_in_minutes = max(delta.total_seconds() // 60, 0)
        
    return difference_in_minutes

边

In [4]:
edges = list(set(zip(without_cancelled['ORIGIN'], without_cancelled['DEST'])))
print(edges[:10])

[('SFO', 'MTJ'), ('TTN', 'PBI'), ('LAS', 'MCI'), ('PIA', 'MSP'), ('AUS', 'SFO'), ('LGB', 'PHX'), ('BNA', 'CHS'), ('ROC', 'JFK'), ('BOS', 'PBI'), ('SLC', 'HNL')]


有向边:起飞延迟，降落延迟，起飞延误数，降落延误数

In [5]:
edge_dep_arr_dic = {}
# counter = 0
for index, row in without_cancelled.iterrows():
    if (row['ORIGIN'], row['DEST']) in edge_dep_arr_dic:
        edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][2] += 1
        if not pd.isna(row['DEP_DELAY_NEW']):
            edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][0] += row['DEP_DELAY_NEW']
            

        # if not pd.isna(row['ARR_DELAY_NEW']):
        #     edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][1] += row['ARR_DELAY_NEW']
        #     edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][3] += 1
        # elif not pd.isna(row['CRS_ARR_TIME']) and not pd.isna(row['ARR_TIME']):
        #     row['ARR_DELAY_NEW'] = time_difference(row['CRS_ARR_TIME'], row['ARR_TIME'])
        #     counter  += 1
        #     edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][1] += row['ARR_DELAY_NEW']
        #     edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][3] += 1

    else:
        edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])] = [0, 0, 0, 0]
        edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][2] += 1
        if not pd.isna(row['DEP_DELAY_NEW']):
            edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][0] += row['DEP_DELAY_NEW']
    

        # if not pd.isna(row['ARR_DELAY_NEW']):
        #     edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][1] += row['ARR_DELAY_NEW']
        #     edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][3] += 1
        # elif not pd.isna(row['CRS_ARR_TIME']) and not pd.isna(row['ARR_TIME']):
        #     row['ARR_DELAY_NEW'] = time_difference(row['CRS_ARR_TIME'], row['ARR_TIME'])
        #     counter +=1
        #     edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][1] += row['ARR_DELAY_NEW']
        #     edge_dep_arr_dic[(row['ORIGIN'], row['DEST'])][3] += 1

print(edge_dep_arr_dic)
# print(counter)

{('IAH', 'MIA'): [33662.0, 0, 2329, 0], ('MIA', 'ATL'): [66582.0, 0, 5517, 0], ('DFW', 'GUC'): [2948.0, 0, 145, 0], ('DFW', 'JAC'): [7908.0, 0, 381, 0], ('GUC', 'DFW'): [2647.0, 0, 142, 0], ('DFW', 'HDN'): [1347.0, 0, 150, 0], ('HDN', 'DFW'): [3374.0, 0, 151, 0], ('JAC', 'ORD'): [10457.0, 0, 497, 0], ('ORD', 'ATL'): [102607.0, 0, 7337, 0], ('ORD', 'JAC'): [6929.0, 0, 521, 0], ('EGE', 'ORD'): [2399.0, 0, 145, 0], ('MIA', 'ORD'): [56300.0, 0, 4701, 0], ('ORD', 'EGE'): [2661.0, 0, 146, 0], ('ORD', 'MIA'): [63409.0, 0, 4689, 0], ('IAH', 'DFW'): [54124.0, 0, 4537, 0], ('MIA', 'IAH'): [35257.0, 0, 2331, 0], ('JAC', 'DFW'): [9306.0, 0, 380, 0], ('DFW', 'IAH'): [46135.0, 0, 4629, 0], ('ATL', 'MIA'): [63751.0, 0, 5508, 0], ('AUS', 'MIA'): [478.0, 0, 50, 0], ('MIA', 'TPA'): [18884.0, 0, 1841, 0], ('TPA', 'MIA'): [17781.0, 0, 1838, 0], ('IAD', 'MIA'): [11412.0, 0, 586, 0], ('MIA', 'IAD'): [9715.0, 0, 585, 0], ('SAT', 'MIA'): [2455.0, 0, 327, 0], ('LAS', 'LAX'): [159444.0, 0, 10973, 0], ('LAX', 'L

转无向，值全部简单相加

In [6]:
def undirected_f(l_dict):
    merged_dict = {}
    for key, value in l_dict.items():
        reverse_key = (key[1], key[0])
        if reverse_key in merged_dict:
            merged_dict[reverse_key] = [sum(x) for x in zip(merged_dict[reverse_key], value)]
        else:
            merged_dict[key] = value
    return merged_dict

edge_dep_arr_dic = undirected_f(edge_dep_arr_dic)
edge_dep_arr_dic

{('IAH', 'MIA'): [68919.0, 0, 4660, 0],
 ('MIA', 'ATL'): [130333.0, 0, 11025, 0],
 ('DFW', 'GUC'): [5595.0, 0, 287, 0],
 ('DFW', 'JAC'): [17214.0, 0, 761, 0],
 ('DFW', 'HDN'): [4721.0, 0, 301, 0],
 ('JAC', 'ORD'): [17386.0, 0, 1018, 0],
 ('ORD', 'ATL'): [195416.0, 0, 14418, 0],
 ('EGE', 'ORD'): [5060.0, 0, 291, 0],
 ('MIA', 'ORD'): [119709.0, 0, 9390, 0],
 ('IAH', 'DFW'): [100259.0, 0, 9166, 0],
 ('AUS', 'MIA'): [1122.0, 0, 100, 0],
 ('MIA', 'TPA'): [36665.0, 0, 3679, 0],
 ('IAD', 'MIA'): [21127.0, 0, 1171, 0],
 ('SAT', 'MIA'): [7230.0, 0, 655, 0],
 ('LAS', 'LAX'): [310787.0, 0, 21945, 0],
 ('DFW', 'MFE'): [24241.0, 0, 1666, 0],
 ('DFW', 'EGE'): [21409.0, 0, 930, 0],
 ('DFW', 'MTJ'): [4797.0, 0, 252, 0],
 ('EGE', 'LAX'): [4203.0, 0, 210, 0],
 ('LAX', 'RDU'): [24294.0, 0, 1584, 0],
 ('LAX', 'AUS'): [77964.0, 0, 6531, 0],
 ('PHL', 'FLL'): [87058.0, 0, 5398, 0],
 ('TPA', 'PHL'): [50878.0, 0, 4748, 0],
 ('CLT', 'SJU'): [10919.0, 0, 1547, 0],
 ('CLT', 'SYR'): [12113.0, 0, 1291, 0],
 ('IAH',

边-延迟dic

In [7]:
edge_delay_dic ={}
for key, value in edge_dep_arr_dic.items():
    edge_delay_dic[key] = value[0] / value[2] #起飞延误

    # try:
    #     edge_delay_dic[key] = value[1] / value[3] #到达延误
    # except:
    #     print(value[1], value[3])
    #     edge_delay_dic[key] = 0

    # edge_delay_dic[key] = (value[0] + value[1]) / (value[2] + value[3])

print(edge_delay_dic)

{('IAH', 'MIA'): 14.789484978540772, ('MIA', 'ATL'): 11.821587301587302, ('DFW', 'GUC'): 19.494773519163765, ('DFW', 'JAC'): 22.62023653088042, ('DFW', 'HDN'): 15.684385382059801, ('JAC', 'ORD'): 17.07858546168959, ('ORD', 'ATL'): 13.553613538632264, ('EGE', 'ORD'): 17.388316151202748, ('MIA', 'ORD'): 12.748562300319488, ('IAH', 'DFW'): 10.938140955705869, ('AUS', 'MIA'): 11.22, ('MIA', 'TPA'): 9.966023375917368, ('IAD', 'MIA'): 18.041844577284373, ('SAT', 'MIA'): 11.038167938931299, ('LAS', 'LAX'): 14.162087035771247, ('DFW', 'MFE'): 14.550420168067227, ('DFW', 'EGE'): 23.02043010752688, ('DFW', 'MTJ'): 19.035714285714285, ('EGE', 'LAX'): 20.014285714285716, ('LAX', 'RDU'): 15.337121212121213, ('LAX', 'AUS'): 11.93752870923289, ('PHL', 'FLL'): 16.12782512041497, ('TPA', 'PHL'): 10.715669755686605, ('CLT', 'SJU'): 7.058177117000646, ('CLT', 'SYR'): 9.382649109217661, ('IAH', 'CLT'): 8.065995975855131, ('PVD', 'CLT'): 7.1771696074900975, ('CHS', 'CLT'): 7.992198581560284, ('PHL', 'PBI')

In [5]:
without_cancelled.isna().any()

FL_DATE                False
OP_UNIQUE_CARRIER      False
TAIL_NUM               False
ORIGIN                 False
DEST                   False
CRS_DEP_TIME           False
DEP_TIME               False
DEP_DELAY_NEW           True
CRS_ARR_TIME           False
ARR_TIME                True
ARR_DELAY_NEW           True
CANCELLED              False
CANCELLATION_CODE       True
CARRIER_DELAY           True
WEATHER_DELAY           True
NAS_DELAY               True
SECURITY_DELAY          True
LATE_AIRCRAFT_DELAY     True
dtype: bool

再处理

In [6]:
without_cancelled.dropna(subset=['TAIL_NUM'], inplace=True)
# without_cancelled.dropna(subset=['ARR_TIME'], inplace=True)
without_cancelled['year'] = pd.to_datetime(without_cancelled['FL_DATE']).dt.year
without_cancelled['month'] = pd.to_datetime(without_cancelled['FL_DATE']).dt.month
without_cancelled['day'] = pd.to_datetime(without_cancelled['FL_DATE']).dt.day
without_cancelled.isna().any()

FL_DATE                False
OP_UNIQUE_CARRIER      False
TAIL_NUM               False
ORIGIN                 False
DEST                   False
CRS_DEP_TIME           False
DEP_TIME               False
DEP_DELAY_NEW           True
CRS_ARR_TIME           False
ARR_TIME                True
ARR_DELAY_NEW           True
CANCELLED              False
CANCELLATION_CODE       True
CARRIER_DELAY           True
WEATHER_DELAY           True
NAS_DELAY               True
SECURITY_DELAY          True
LATE_AIRCRAFT_DELAY     True
year                   False
month                  False
day                    False
dtype: bool

延误传播网络

In [7]:
edge_delay_dic ={}

sorted_df = without_cancelled.sort_values(by=['TAIL_NUM', 'year', 'month', 'day', 'CRS_DEP_TIME']).reset_index(drop=True)
# counter = 0
# match_rows = []
for index, row in sorted_df.iterrows():
    if index < len(sorted_df) - 1:
        next_row = sorted_df.iloc[index + 1]

        if pd.isna(row['ARR_TIME']) or pd.isna(next_row['ARR_TIME']):
            continue
        # 默认第一个ARR_DELAY_NEW不会为空
        # elif pd.isna(row['ARR_DELAY_NEW']):
        #     row['ARR_DELAY_NEW'] = time_difference(row['FL_DATE'], row['CRS_ARR_TIME'], row['FL_DATE'], row['ARR_TIME'])
        elif pd.isna(next_row['ARR_DELAY_NEW']):
            sorted_df.loc[index+1, ['ARR_DELAY_NEW']] = time_difference(next_row['FL_DATE'], next_row['CRS_ARR_TIME'], next_row['FL_DATE'], next_row['ARR_TIME'])
            
        if row['TAIL_NUM'] == next_row['TAIL_NUM'] and row['DEST'] == next_row['ORIGIN']:
            # counter +=1
            # tat = time_difference(row['FL_DATE'], row['CRS_ARR_TIME'], next_row['FL_DATE'], next_row['CRS_DEP_TIME']) - time_difference(row['FL_DATE'], row['ARR_TIME'],next_row['FL_DATE'], next_row['DEP_TIME'])
            # if tat>0 and time_difference(next_row['FL_DATE'], next_row['CRS_ARR_TIME'], next_row['FL_DATE'], next_row['ARR_TIME'])>0:
            tat = row['ARR_DELAY_NEW'] - next_row['DEP_DELAY_NEW']
            if tat>0 and next_row['ARR_DELAY_NEW']>0:
                # if ((next_row['ORIGIN'], next_row['DEST']) == ('LAX', 'CLE')):
                #     match_rows.append(row)
                #     match_rows.append(next_row)
                    
                if (next_row['ORIGIN'], next_row['DEST']) in edge_delay_dic:
                    edge_delay_dic[(next_row['ORIGIN'], next_row['DEST'])] += 1
                else:
                    edge_delay_dic[(next_row['ORIGIN'], next_row['DEST'])] = 1

print(edge_delay_dic[('LAX', 'CLE')])
# combined_row = pd.concat(match_rows, axis=1).T
# pd.set_option('display.max_columns', None)
# print(combined_row)

# print(counter)

121


networkx转换格式

In [8]:
def f(dic):
    tuple_list = []
    for key, value in dic.items():
        tuple_list.append((key[0], key[1], {'weight': value}))
    return tuple_list

edge_list = f(edge_delay_dic)
print(edge_list[:10])

[('ORD', 'MIA', {'weight': 411}), ('MIA', 'SAT', {'weight': 21}), ('MIA', 'IAH', {'weight': 198}), ('MIA', 'AUS', {'weight': 4}), ('ATL', 'MIA', {'weight': 422}), ('MIA', 'DFW', {'weight': 338}), ('EGE', 'DFW', {'weight': 85}), ('MTJ', 'DFW', {'weight': 19}), ('EGE', 'LAX', {'weight': 12}), ('MIA', 'TPA', {'weight': 170})]


In [ ]:
# G = nx.Graph()

# G.add_edges_from(edge_list)
# G.add_nodes_from(without_cancelled['ORIGIN'])

# # 绘制图形
# nx.draw(G, nx.random_layout(G), with_labels=True, width = 0.1)

# # 显示图形
# # plt.show()

csv转换格式

In [9]:
def rf(dic):
    tuple_list = []
    for key, value in dic.items():
        tuple_list.append((key[0], key[1], value))
    return tuple_list

csv_list = rf(edge_delay_dic)

平均延误网络保存

In [ ]:
df_edges = pd.DataFrame(csv_list, columns=['source', 'target', 'weight'])
df_edges_sorted = df_edges.sort_values(by=['source', 'target', 'weight'])
df_edges_sorted.to_csv('../建网结果/new_2017_美国_平均.csv', index=False)
print('done')

done


kz的ATL到ABE不一样

延误传播网络保存

In [ ]:
df_edges = pd.DataFrame(csv_list, columns=['source', 'target', 'weight'])
df_edges_sorted = df_edges.sort_values(by=['source', 'target', 'weight'])
df_edges_sorted.to_csv('../建网结果/new_2017_美国_传播.csv', index=False)
print('done')

done


LAX到CLE的延误传播应该是1.前一班航班的降落机场应该和后一班的起飞机场一致